# Tugas Klasifikasi Kelompok 7
**Data Mining and Data Warehouse**

- Ade Mulyana (2108799)
- Muhammad Alam Basalamah (2101677)
- Muhammad Rizki (2107922)
- Muhammad Yusuf Bahtiar (2107980)
- Rasyid Andriansyah (2101963)

## Spesifikasi Soal

### Melakukan Praproses dan Prediksi
### Meningkatkan F1-Score dari dataset yang ada

In [1]:
# Import library yang dibutuhkan
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from datetime import datetime,date
from time import strptime
from sklearn import preprocessing 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
pd.set_option('display.max_columns', 500)

# Import warnings filter
from warnings import simplefilter

E:\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Import data
data_ori =pd.read_csv('data_return\orders_train.txt', delimiter=";")


# Menampilkan data teratas di dalam data frame
print(list(data_ori))



['orderItemID', 'orderDate', 'deliveryDate', 'itemID', 'size', 'color', 'manufacturerID', 'price', 'customerID', 'salutation', 'dateOfBirth', 'state', 'creationDate', 'returnShipment']


# PRAPROSES

In [3]:
# Mengecek value yang unik di dalam data frame
data_ori.nunique()

orderItemID       481092
orderDate            365
deliveryDate         328
itemID              3007
size                 122
color                 88
manufacturerID       165
price                379
customerID         59754
salutation             5
dateOfBirth        14309
state                 16
creationDate         775
returnShipment         2
dtype: int64

In [4]:
# Mengecek value yang bernilai NULL 
print(data_ori.isnull().sum())

orderItemID       0
orderDate         0
deliveryDate      0
itemID            0
size              0
color             0
manufacturerID    0
price             0
customerID        0
salutation        0
dateOfBirth       0
state             0
creationDate      0
returnShipment    0
dtype: int64


### Membuat beberapa kolom tambahan

In [5]:
# Mengubah tipe data
data_ori.head(6)
pd.Timestamp.max
data_ori["dateOfBirth"] = pd.to_datetime(data_ori["dateOfBirth"], errors = 'coerce')

In [6]:
# Data yang bervalue "?" diubah menjadi null karena tidak bisa di tebak
data_ori.deliveryDate=data_ori.deliveryDate.replace("?", np.nan)
data_ori["color"]=data_ori["color"].replace("?","other")
data_ori.dateOfBirth=data_ori.dateOfBirth.replace("?", np.nan)

In [7]:
# Mengecek nilai null pada setiap kolom 
print(data_ori.isnull().sum())

orderItemID           0
orderDate             0
deliveryDate      39419
itemID                0
size                  0
color                 0
manufacturerID        0
price                 0
customerID            0
salutation            0
dateOfBirth       48892
state                 0
creationDate          0
returnShipment        0
dtype: int64


In [8]:
# Mengubah tipe data
data_ori["deliveryDate"] = pd.to_datetime(data_ori["deliveryDate"])
data_ori["orderDate"] = pd.to_datetime(data_ori["orderDate"])
data_ori["dateOfBirth"] = pd.to_datetime(data_ori["dateOfBirth"])
data_ori["creationDate"] = pd.to_datetime(data_ori["creationDate"])

# Membuat kolom deliverytime_days yang berisi hari di mana barang dikirimkan
data_ori["deliverytime_days"] = (data_ori["deliveryDate"] - data_ori["orderDate"])/np.timedelta64(1, 'D')

# Membuat kolom age_at_order_yrs yang berisi umur pembeli ketika mengorder barang
data_ori["age_at_order_yrs"] = (data_ori["orderDate"] - data_ori["dateOfBirth"])/np.timedelta64(1, 'Y')

# Membuat kolom product_age_days yang berisi umur barang ketika sampai di pembeli
data_ori["product_age_days"] = (data_ori["orderDate"] - data_ori["creationDate"])/np.timedelta64(1, 'D')

In [9]:
# Menghapus value yang hilang
data_ori.dropna(subset = ['deliveryDate'])
data_ori.dropna(subset = ['dateOfBirth'])

# Mengonversi value dari atribut orderDate menjadi hari ke dalam atribut day_of_week
data_ori['day_of_week'] = data_ori['orderDate'].dt.day_name()

# # Mengonversi value dari atribut orderDate menjadi bulan
# data_ori['month'] = data_ori['orderDate'].dt.month_name()


# season_dict = {'January': 'Winter',
#                'February': 'Winter',
#                'March': 'Spring', 
#                'April': 'Spring',
#                'May': 'Spring',
#                'June': 'Summer',
#                'July': 'Summer',
#                'August': 'Summer',
#                'September': 'Fall',
#                'October': 'Fall',
#                'November': 'Fall',
#                'December': 'Winter'}
# data_ori['Season'] = data_ori['month'].apply(lambda x: season_dict[x])

data_ori.head()

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,dateOfBirth,state,creationDate,returnShipment,deliverytime_days,age_at_order_yrs,product_age_days,day_of_week
0,1,2012-04-01,2012-04-03,186,m,denim,25,69.90,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,0,2.0,47.234372,342.0,Sunday
1,2,2012-04-01,2012-04-03,71,9+,ocher,21,69.95,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,1,2.0,47.234372,342.0,Sunday
2,3,2012-04-01,2012-04-03,71,9+,curry,21,69.95,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,1,2.0,47.234372,342.0,Sunday
3,4,2012-04-02,NaT,22,m,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0,NaN,52.398064,89.0,Monday
4,5,2012-04-02,1990-12-31,151,39,black,53,29.90,825,Mrs,1964-07-11,Rhineland-Palatinate,2011-02-16,0,-7763.0,47.727195,411.0,Monday


In [10]:
#Mendapatkan ratio return barang per customer
n_return=data_ori.groupby('customerID')['returnShipment'].transform('sum') 
total_return=data_ori.groupby('customerID')['returnShipment'].transform('count')
data_ori['Return_ratio'] = n_return/ total_return

#Import data
data_test =pd.read_csv('data_bersih_tes.csv')
data_test['Return_ratio'] = n_return/ total_return

In [11]:
# Menghapus kolom orderDate, deliveryDate, dateOfBirth, dan creationDate
data_ori = data_ori.drop("orderDate", axis=1)
data_ori = data_ori.drop("deliveryDate", axis=1)
data_ori = data_ori.drop("dateOfBirth", axis=1)
data_ori = data_ori.drop("creationDate", axis=1)

data_ori.head()

,orderItemID,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,deliverytime_days,age_at_order_yrs,product_age_days,day_of_week,Return_ratio
0,1,186,m,denim,25,69.90,794,Mrs,Baden-Wuerttemberg,0,2.0,47.234372,342.0,Sunday,0.454545
1,2,71,9+,ocher,21,69.95,794,Mrs,Baden-Wuerttemberg,1,2.0,47.234372,342.0,Sunday,0.454545
2,3,71,9+,curry,21,69.95,794,Mrs,Baden-Wuerttemberg,1,2.0,47.234372,342.0,Sunday,0.454545
3,4,22,m,green,14,39.90,808,Mrs,Saxony,0,NaN,52.398064,89.0,Monday,0.000000
4,5,151,39,black,53,29.90,825,Mrs,Rhineland-Palatinate,0,-7763.0,47.727195,411.0,Monday,0.111111


In [12]:
# Menghapus value yang bernilai negatif karena tidak masuk akal
data_ori = data_ori[data_ori['deliverytime_days'] >= 0]
data_ori = data_ori[data_ori["age_at_order_yrs"] >= 0]
data_ori = data_ori[data_ori["product_age_days"] >= 0]

data_ori.describe()

,orderItemID,itemID,manufacturerID,price,customerID,returnShipment,deliverytime_days,age_at_order_yrs,product_age_days,Return_ratio
count,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000,392430.000000
mean,241312.310432,1407.349869,29.853627,69.904607,32345.617534,0.528385,10.640007,48.569169,311.806597,0.494104
std,138650.814789,928.214725,30.164870,45.259346,23555.480355,0.499194,17.916882,11.003492,257.657294,0.261008
min,1.000000,1.000000,1.000000,0.000000,6.000000,0.000000,0.000000,0.082137,0.000000,0.000000
25%,122736.500000,343.000000,5.000000,34.900000,12348.000000,0.000000,2.000000,42.514220,21.000000,0.320000
50%,240399.500000,1575.000000,23.000000,59.900000,27327.500000,1.000000,3.000000,47.787429,310.000000,0.500000
75%,361159.750000,2344.000000,43.000000,89.900000,49224.000000,1.000000,8.000000,53.413828,533.000000,0.680000
max,481088.000000,3070.000000,166.000000,999.000000,86609.000000,1.000000,175.000000,112.363704,774.000000,1.000000


### Mentransformasikan value dari atribut Size dan Color

In [13]:
# Mengecek value yang terdapat di dalam atribut Size
data_ori["size"].unique()

array(['m', '9+', 'xxl', '39', '43', '37', '38', 'l', 'xl', '42', '41',
       'unsized', 's', '10+', '40', '36', '152', '35', '34', '8+', '9',
       '46', '6', '25', '20', '5', '10', '42+', '44', '8', '3', '6+',
       '48', '7+', '22', '12', '45', '7', '24', '39+', '27', '32', '4+',
       '11', '26', '40+', '19', '21', '5+', '116', '2', '28', '11+',
       '36+', '164', '4', '33', '29', '50', '18', '41+', '1', '38+', '30',
       '31', '104', '128', '95', '37+', '3+', '140', '23', '13', 'S',
       '44+', 'xxxl', '54', '52', '47', '3434', '49', '84', '3332', '56',
       '13+', '76', '85', '176', '88', '45+', '14', 'L', '46+', '43+',
       '3632', '90', '3832', '3634', '4032', '3432', 'xs', '80', '2+',
       '100', '3132', '58', '4034', '3834', '12+', '2932', '110', '122',
       'XXL', 'M', 'XL', 'XXXL', '4232', 'XS', '92', '96', '3334', '105'],
      dtype=object)

In [14]:
# Mengonversi value dari atribut size yang kemungkinan barangnya adalah pakaian
data_ori["size"]= data_ori["size"].replace("S",33)
data_ori["size"]= data_ori["size"].replace("M",37)
data_ori["size"]= data_ori["size"].replace("L",41)
data_ori["size"]= data_ori["size"].replace("XL",45)
data_ori["size"]= data_ori["size"].replace("XXL",49)
data_ori["size"]= data_ori["size"].replace("XXXL",51)
data_ori["size"]= data_ori["size"].replace("xxxl",51)
data_ori["size"]= data_ori["size"].replace("s",33)
data_ori["size"]= data_ori["size"].replace("m",37)
data_ori["size"]= data_ori["size"].replace("l",41)
data_ori["size"]= data_ori["size"].replace("xl",45)
data_ori["size"]= data_ori["size"].replace("xxl",49)
data_ori["size"]= data_ori["size"].replace("S",33)
data_ori["size"]= data_ori["size"].replace("M",37)
data_ori["size"]= data_ori["size"].replace("L",41)
data_ori["size"]= data_ori["size"].replace("XL",45)
data_ori["size"]= data_ori["size"].replace("XXL",49)
data_ori["size"]= data_ori["size"].replace("XXXL",51)
data_ori["size"]= data_ori["size"].replace("xxxl",51)
data_ori["size"]= data_ori["size"].replace("xs",30)
data_ori["size"]= data_ori["size"].replace("XS",30)

In [15]:
# Mengonversi value dari atribut size yang kemungkinan barangnya adalah sepatu
data_ori["size"]= data_ori["size"].replace("10+",10)
data_ori["size"]= data_ori["size"].replace("11+",11)
data_ori["size"]= data_ori["size"].replace("12+",12)
data_ori["size"]= data_ori["size"].replace("2+",2)
data_ori["size"]= data_ori["size"].replace("3+",3)
data_ori["size"]= data_ori["size"].replace("36+",36)
data_ori["size"]= data_ori["size"].replace("37+",37)
data_ori["size"]= data_ori["size"].replace("38+",38)
data_ori["size"]= data_ori["size"].replace("39+",39)
data_ori["size"]= data_ori["size"].replace("40+",40)
data_ori["size"]= data_ori["size"].replace("41+",41)
data_ori["size"]= data_ori["size"].replace("42+",42)
data_ori["size"]= data_ori["size"].replace("43+",43)
data_ori["size"]= data_ori["size"].replace("44+",44)
data_ori["size"]= data_ori["size"].replace("45+",45)
data_ori["size"]= data_ori["size"].replace("46+",46)
data_ori["size"]= data_ori["size"].replace("5+",5)
data_ori["size"]= data_ori["size"].replace("6+",6)
data_ori["size"]= data_ori["size"].replace("7+",7)
data_ori["size"]= data_ori["size"].replace("8+",8)
data_ori["size"]= data_ori["size"].replace("9+",9)
data_ori["size"]= data_ori["size"].replace("4+",4)
data_ori["size"]= data_ori["size"].replace("13+",13)

In [16]:
data_ori["size"].unique()

array([37, 9, 49, '39', '43', '37', '38', 41, 45, '42', '41', 'unsized',
       33, 10, '40', '36', '152', '35', '34', 8, '9', '46', '6', '25',
       '20', '5', '10', 42, '44', '8', '3', 6, '48', 7, '22', '12', '45',
       '7', '24', 39, '27', '32', 4, '11', '26', 40, '19', '21', 5, '116',
       '2', '28', 11, 36, '164', '4', '33', '29', '50', '18', '1', 38,
       '30', '31', '104', '128', '95', 3, '140', '23', '13', 44, 51, '54',
       '52', '47', '3434', '49', '84', '3332', '56', 13, '76', '85',
       '176', '88', '14', 46, 43, '3632', '90', '3832', '3634', '4032',
       '3432', 30, '80', 2, '100', '3132', '58', '4034', '3834', 12,
       '2932', '110', '122', '4232', '92', '96', '3334', '105'],
      dtype=object)

In [17]:
# Menghapus data yang memiliki value "Unsized" dari atribut size
data_ori = data_ori[data_ori["size"] != "unsized"]

In [18]:
# Mengecek value yang terdapat di dalam atribut Color
data_ori["color"].unique()

array(['denim', 'ocher', 'curry', 'brown', 'red', 'green', 'mocca',
       'black', 'anthracite', 'olive', 'petrol', 'blue', 'grey', 'beige',
       'ecru', 'turquoise', 'magenta', 'purple', 'pink', 'khaki', 'navy',
       'habana', 'silver', 'white', 'stained', 'orange', 'azure',
       'apricot', 'nature', 'berry', 'ash', 'hibiscus', 'fuchsia', 'blau',
       'dark denim', 'mint', 'mango', 'ivory', 'yellow', 'bordeaux',
       'ancient', 'aquamarine', 'brwon', 'aubergine', 'almond', 'aqua',
       'dark garnet', 'avocado', 'creme', 'champagner', 'cortina mocca',
       'currant purple', 'cognac', 'aviator', 'gold', 'ebony',
       'cobalt blue', 'baltic blue', 'kanel', 'curled', 'antique pink',
       'dark grey', 'darkblue', 'copper coin', 'terracotta', 'basalt',
       'caramel', 'coral', 'jade', 'opal', 'amethyst', 'striped',
       'mahagoni', 'floral', 'dark navy', 'dark oliv', 'vanille',
       'ingwer', 'iron', 'graphite', 'leopard', 'oliv', 'bronze',
       'crimson', 'perlmu

In [19]:
# Membuat klasterisasi warna pada atribut Color
data_ori["color"]=data_ori["color"].replace("berry","red")
data_ori["color"]=data_ori["color"].replace("bordeaux","red")
data_ori["color"]=data_ori["color"].replace("dark garnet","red")
data_ori["color"]=data_ori["color"].replace("fuchsia","red")
data_ori["color"]=data_ori["color"].replace("crimson","red")
data_ori["color"]=data_ori["color"].replace("magenta","pink")
data_ori["color"]=data_ori["color"].replace("antique pink","pink")
data_ori["color"]=data_ori["color"].replace("pink","pink")
data_ori["color"]=data_ori["color"].replace("pallid","pink")
data_ori["color"]=data_ori["color"].replace("coral","pink")
data_ori["color"]=data_ori["color"].replace("hibiscus","red")
data_ori["color"]=data_ori["color"].replace("orange","orange")
data_ori["color"]=data_ori["color"].replace("apricot","orange")
data_ori["color"]=data_ori["color"].replace("ocher","yellow")
data_ori["color"]=data_ori["color"].replace("beige","yellow")
data_ori["color"]=data_ori["color"].replace("lemon","yellow")
data_ori["color"]=data_ori["color"].replace("vanille","yellow")
data_ori["color"]=data_ori["color"].replace("creme","yellow")
data_ori["color"]=data_ori["color"].replace("almond","yellow")
data_ori["color"]=data_ori["color"].replace("mango","yellow")
data_ori["color"]=data_ori["color"].replace("curry","yellow")
data_ori["color"]=data_ori["color"].replace("ancient","yellow")
data_ori["color"]=data_ori["color"].replace("ingwer","yellow")
data_ori["color"]=data_ori["color"].replace("gold","yellow")
data_ori["color"]=data_ori["color"].replace("oliv","green")
data_ori["color"]=data_ori["color"].replace("aquamarine","green")
data_ori["color"]=data_ori["color"].replace("nature olive","green")
data_ori["color"]=data_ori["color"].replace("nature","green")
data_ori["color"]=data_ori["color"].replace("avocado","green")
data_ori["color"]=data_ori["color"].replace("khaki","green")
data_ori["color"]=data_ori["color"].replace("mint","green")
data_ori["color"]=data_ori["color"].replace("jade","green")
data_ori["color"]=data_ori["color"].replace("gold","green")
data_ori["color"]=data_ori["color"].replace("olive","green")
data_ori["color"]=data_ori["color"].replace("dark oliv","green")
data_ori["color"]=data_ori["color"].replace("petrol","blue")
data_ori["color"]=data_ori["color"].replace("denim turquoise","blue")
data_ori["color"]=data_ori["color"].replace("turquoise","blue")
data_ori["color"]=data_ori["color"].replace("denim","blue")
data_ori["color"]=data_ori["color"].replace("dark denim","blue")
data_ori["color"]=data_ori["color"].replace("azure","blue")
data_ori["color"]=data_ori["color"].replace("navy","blue")
data_ori["color"]=data_ori["color"].replace("blau","blue")
data_ori["color"]=data_ori["color"].replace("aqua", "blue")
data_ori["color"]=data_ori["color"].replace("darkblue", "blue")
data_ori["color"]=data_ori["color"].replace("dark navy","blue")
data_ori["color"]=data_ori["color"].replace("baltic blue","blue")
data_ori["color"]=data_ori["color"].replace("cobalt blue","blue")
data_ori["color"]=data_ori["color"].replace("aubergine","purple")
data_ori["color"]=data_ori["color"].replace("amethyst","purple")
data_ori["color"]=data_ori["color"].replace("currant purple","purple")
data_ori["color"]=data_ori["color"].replace("ebony","black")
data_ori["color"]=data_ori["color"].replace("anthracite","grey")
data_ori["color"]=data_ori["color"].replace("ash","grey")
data_ori["color"]=data_ori["color"].replace("iron","grey")
data_ori["color"]=data_ori["color"].replace("dark grey","grey")
data_ori["color"]=data_ori["color"].replace("basalt","grey")
data_ori["color"]=data_ori["color"].replace("graphite","grey")
data_ori["color"]=data_ori["color"].replace("silver","grey")
data_ori["color"]=data_ori["color"].replace("stained","brown")
data_ori["color"]=data_ori["color"].replace("mocha","brown")
data_ori["color"]=data_ori["color"].replace("ecru","brown")
data_ori["color"]=data_ori["color"].replace("brwon","brown")
data_ori["color"]=data_ori["color"].replace("terracotta","brown")
data_ori["color"]=data_ori["color"].replace("bronze","brown")
data_ori["color"]=data_ori["color"].replace("cortina mocca","brown")
data_ori["color"]=data_ori["color"].replace("caramel","brown")
data_ori["color"]=data_ori["color"].replace("copper coin","brown")
data_ori["color"]=data_ori["color"].replace("cognac","brown")
data_ori["color"]=data_ori["color"].replace("mahagoni","brown")
data_ori["color"]=data_ori["color"].replace("habana","brown")
data_ori["color"]=data_ori["color"].replace("mocca","brown")
data_ori["color"]=data_ori["color"].replace("ivory","white")
data_ori["color"]=data_ori["color"].replace("champagner","white")
data_ori["color"]=data_ori["color"].replace("kanel","white")
data_ori["color"]=data_ori["color"].replace("perlmutt","white")
data_ori["color"]=data_ori["color"].replace("leopard","other")
data_ori["color"]=data_ori["color"].replace("opal","other")
data_ori["color"]=data_ori["color"].replace("curled","other")
data_ori["color"]=data_ori["color"].replace("aviator","other")
data_ori["color"]=data_ori["color"].replace("striped","other")
data_ori["color"]=data_ori["color"].replace("floral","other")

In [20]:
# Mengubah tipe data dari value atribut color menjadi string
data_ori['color'] = data_ori.color.astype(str)

data_ori['color'].describe()
data_ori['color'].value_counts()

black     68966
blue      68524
grey      48308
brown     44023
green     42400
red       38738
purple    17468
white     15627
yellow    14542
pink       8456
orange     2865
other       726
Name: color, dtype: int64

In [21]:
# Menampilkan 30 data teratas di dalam data frame
data_ori.head(9)

,orderItemID,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,deliverytime_days,age_at_order_yrs,product_age_days,day_of_week,Return_ratio
0,1,186,37,blue,25,69.90,794,Mrs,Baden-Wuerttemberg,0,2.0,47.234372,342.0,Sunday,0.454545
1,2,71,9,yellow,21,69.95,794,Mrs,Baden-Wuerttemberg,1,2.0,47.234372,342.0,Sunday,0.454545
2,3,71,9,yellow,21,69.95,794,Mrs,Baden-Wuerttemberg,1,2.0,47.234372,342.0,Sunday,0.454545
7,8,32,49,brown,3,21.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000
8,9,32,49,red,3,21.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000
9,10,57,49,green,3,39.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000
10,11,2,49,brown,2,39.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000
11,12,259,39,black,1,119.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000
12,13,603,39,black,55,169.90,850,Mrs,North Rhine-Westphalia,1,1.0,63.984887,411.0,Monday,1.000000


In [22]:
# Mengubah tipe data dari value atribut day_of_week menjadi string
data_ori['price'] = data_ori.price.astype(int)
data_ori['deliverytime_days'] = data_ori.deliverytime_days.astype(int)
data_ori['age_at_order_yrs'] = data_ori.age_at_order_yrs.astype(int)
data_ori['product_age_days'] = data_ori.product_age_days.astype(int)
data_ori['day_of_week'] = data_ori.day_of_week.astype(str)
# data_ori['month'] = data_ori.month.astype(str)
# data_ori['Season'] = data_ori.Season.astype(str)
data_ori.dtypes

orderItemID            int64
itemID                 int64
size                  object
color                 object
manufacturerID         int64
price                  int32
customerID             int64
salutation            object
state                 object
returnShipment         int64
deliverytime_days      int32
age_at_order_yrs       int32
product_age_days       int32
day_of_week           object
Return_ratio         float64
dtype: object

### One Hot Encoding Untuk merepresentasikan biner pada variabel string

In [23]:
# Proses One Hot Encoding untuk atribut color, salutation, state, dan day_of_week
data_ori = pd.get_dummies(data=data_ori, columns=['color', 'salutation','state', 'day_of_week'])
# data_ori = pd.get_dummies(data=data_ori, columns=['color', 'salutation','state','Season'])
data_ori.drop(['orderItemID'], axis=1,inplace=True)
data_ori.head()

,itemID,size,manufacturerID,price,customerID,returnShipment,deliverytime_days,age_at_order_yrs,product_age_days,Return_ratio,color_black,color_blue,color_brown,color_green,color_grey,color_orange,color_other,color_pink,color_purple,color_red,color_white,color_yellow,salutation_Company,salutation_Family,salutation_Mr,salutation_Mrs,salutation_not reported,state_Baden-Wuerttemberg,state_Bavaria,state_Berlin,state_Brandenburg,state_Bremen,state_Hamburg,state_Hesse,state_Lower Saxony,state_Mecklenburg-Western Pomerania,state_North Rhine-Westphalia,state_Rhineland-Palatinate,state_Saarland,state_Saxony,state_Saxony-Anhalt,state_Schleswig-Holstein,state_Thuringia,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,186,37,25,69,794,0,2,47,342,0.454545,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,71,9,21,69,794,1,2,47,342,0.454545,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,71,9,21,69,794,1,2,47,342,0.454545,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7,32,49,3,21,850,1,1,63,411,1.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
8,32,49,3,21,850,1,1,63,411,1.000000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0


In [24]:
#menyiapkan data train dan test, cara pertama adalah dengan teknik hold out. 
from sklearn.model_selection import train_test_split
X = data_ori.drop(['returnShipment'],axis=1) #training features
Y = data_ori['returnShipment'] # target variable
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, random_state=101)

# Menyimpan nama kolom untuk keperluan prediksi nanti
import pickle
with open('x_train_columns.pickle', 'wb') as fp:
    pickle.dump(X_train.columns, fp)

In [25]:
# Proses Normalisasi
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [26]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)
X_test_minmax = mm_scaler.transform(X_test)


In [27]:
# StandardScaler
# StandardScaler mengubah distribusi sehingga nilai rata-rata menjadi nol dan standar deviasi 1.

# y = (x – mean) / standard_deviation

ss_scaler = preprocessing.StandardScaler()
X_train_ss = mm_scaler.fit_transform(X_train)
X_test_ss = mm_scaler.transform(X_test)


In [28]:
# Kompresi ke dimensi yang lebih rendah
# Setelah menggunakan one-hot encoding untuk atribut kategorik (termasuk Event), maka jumlah atribut menjadi besar (784 atribut). Reduksi dimensi dapat digunakan untuk mengatasi ini. Selain mempercepat proses training, untuk visualisasi, dan dalam beberapa kasus dapat meningkatkan kinerja model.

# PCA (Principal Component Analysis)  dapat digunakan untuk mereduksi dimensi.  

# Scaling perlu dilakukan terlebih dulu:


from sklearn.decomposition import PCA
ss_scaler = preprocessing.StandardScaler()
X_train_ss = ss_scaler.fit_transform(X_train) #fit hanya di train
X_test_ss = ss_scaler.transform(X_test)
pca = PCA(.75)
X_train_pca = pca.fit_transform(X_train_ss)
X_test_pca = pca.transform(X_test_ss)


## Modeling

## Modeling Naive Bayes

In [29]:
## Pembentukan Model Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.691699604743083
              precision    recall  f1-score   support

           0       0.71      0.55      0.62     33743
           1       0.68      0.81      0.74     40386

    accuracy                           0.69     74129
   macro avg       0.69      0.68      0.68     74129
weighted avg       0.69      0.69      0.69     74129



### f1 score sudah cukup baik, tetapi kita coba dahulu metode lainnya

## Modeling Decision Tree

In [30]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))


Akurasi 0.6589189116270285
              precision    recall  f1-score   support

           0       0.62      0.63      0.63     33743
           1       0.69      0.68      0.69     40386

    accuracy                           0.66     74129
   macro avg       0.66      0.66      0.66     74129
weighted avg       0.66      0.66      0.66     74129



### Dalam metode Decision tree dapat dilihat f1 score mengalami penurunan dari metode sebelumnya

## Modeling Random Forest

In [31]:
##Pembentukan model random forest
clf =RandomForestClassifier(n_estimators=100, random_state=100)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))


Akurasi 0.7181399991906001
              precision    recall  f1-score   support

           0       0.71      0.64      0.67     33743
           1       0.72      0.78      0.75     40386

    accuracy                           0.72     74129
   macro avg       0.72      0.71      0.71     74129
weighted avg       0.72      0.72      0.72     74129



### Modeling Random Forest mengalami peningkatan f1 score dibanding modeling naive bayes, dan decision tree

## Modeling XGBOOST

In [32]:
import xgboost as xgb
clf = xgb.XGBClassifier(learning_rate =0.01,
 n_estimators=100,
 max_depth=15,
 min_child_weight=7,
 gamma=0.4,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
# num_class adalah jumlah kelas
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))


E:\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:50:09] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Akurasi 0.7388741248364338
              precision    recall  f1-score   support

           0       0.76      0.63      0.69     33743
           1       0.73      0.83      0.78     40386

    accuracy                           0.74     74129
   macro avg       0.74      0.73      0.73     74129
weighted avg       0.74      0.74      0.74     74129



## Karena f1 score yang paling besar adalah XGBOOST maka kita pakai XGBOOST untuk di import

In [33]:
## melakukan export modelling
import joblib
import pandas as pd
joblib.dump(clf, 'rxgboost_order.joblib')


['rxgboost_order.joblib']

In [34]:
data_test.drop(['Unnamed: 0'], axis=1,inplace=True)



In [35]:
data_test = pd.get_dummies(data=data_test, columns=['color', 'salutation','state', 'day_of_week'])
data_test.head()

,orderItemID,itemID,size,manufacturerID,price,customerID,deliverytime_days,age_at_order_yrs,product_age_days,month,Season,Return_ratio,color_black,color_blue,color_brown,color_green,color_grey,color_orange,color_other,color_pink,color_purple,color_red,color_white,color_yellow,salutation_Company,salutation_Family,salutation_Mr,salutation_Mrs,salutation_not reported,state_Baden-Wuerttemberg,state_Bavaria,state_Berlin,state_Brandenburg,state_Bremen,state_Hamburg,state_Hesse,state_Lower Saxony,state_Mecklenburg-Western Pomerania,state_North Rhine-Westphalia,state_Rhineland-Palatinate,state_Saarland,state_Saxony,state_Saxony-Anhalt,state_Schleswig-Holstein,state_Thuringia,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,1,2347,43,1,89,12489,2,49,343,April,Spring,0.454545,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2,2741,43,1,99,12489,2,49,343,April,Spring,0.454545,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,3,2514,9,19,79,12489,2,49,343,April,Spring,0.454545,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,2347,42,1,89,12489,35,49,343,April,Spring,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,6,2318,41,1,89,3649,1,57,0,April,Spring,0.111111,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [36]:
#import data train
import pickle
with open ('x_train_columns.pickle', 'rb') as fp:
    X_train_column = list(pickle.load(fp))

In [37]:
data_test = data_test[X_train_column]

In [38]:
#Memanggil Modeling
import joblib
clf = joblib.load("rxgboost_order.joblib")
clf.predict(data_test)


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [39]:
data_test.head()

,itemID,size,manufacturerID,price,customerID,deliverytime_days,age_at_order_yrs,product_age_days,Return_ratio,color_black,color_blue,color_brown,color_green,color_grey,color_orange,color_other,color_pink,color_purple,color_red,color_white,color_yellow,salutation_Company,salutation_Family,salutation_Mr,salutation_Mrs,salutation_not reported,state_Baden-Wuerttemberg,state_Bavaria,state_Berlin,state_Brandenburg,state_Bremen,state_Hamburg,state_Hesse,state_Lower Saxony,state_Mecklenburg-Western Pomerania,state_North Rhine-Westphalia,state_Rhineland-Palatinate,state_Saarland,state_Saxony,state_Saxony-Anhalt,state_Schleswig-Holstein,state_Thuringia,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,2347,43,1,89,12489,2,49,343,0.454545,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2741,43,1,99,12489,2,49,343,0.454545,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2514,9,19,79,12489,2,49,343,0.454545,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,2347,42,1,89,12489,35,49,343,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2318,41,1,89,3649,1,57,0,0.111111,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


## Melakukan Prediksi

In [40]:
data_final = pd.DataFrame()
data_final['orderItemID'] = data_test['itemID']
data_final['prediction'] = clf.predict(data_test)

In [41]:
data_final.head()

,orderItemID,prediction
0,2347,1
1,2741,1
2,2514,1
3,2347,1
4,2318,1


In [42]:
data_final.to_csv('data_return\order_class_final_banget.txt', sep=';', index=False)